# RAG System Demo

This notebook demonstrates how to use the Retrieval-Augmented Generation (RAG) system with the following features:

1. Adding documents to the knowledge base
2. Processing documents from various sources (files, URLs)
3. Searching the knowledge base
4. Asking questions with RAG-enhanced responses

The system handles document chunking, embedding generation, vector storage, and retrieval automatically.

## Setup and Imports

In [ ]:
import asyncio
import os
import sys
from pathlib import Path

# Add project root to sys.path if needed
project_root = Path("..")
if str(project_root.absolute()) not in sys.path:
    sys.path.append(str(project_root.absolute()))

# Import RAG components
from src.backend.core.rag_document_processor import rag_document_processor
from src.backend.core.enhanced_vector_store import enhanced_vector_store
from src.backend.agents.enhanced_rag_agent import EnhancedRAGAgent

# Create RAG agent instance
rag_agent = EnhancedRAGAgent("notebook-demo-agent")

## 1. Adding Individual Documents

You can add documents directly as text strings with metadata:

In [ ]:
# Example document as text
document_text = """
# Vector Databases in RAG Systems

Vector databases are specialized databases designed to store and efficiently query vector embeddings.
They are a critical component in RAG systems as they enable fast similarity search for retrieving
relevant context when answering user queries.

Popular vector database options include:
- FAISS: Facebook AI Similarity Search, an efficient open-source library
- Pinecone: A fully managed vector database service
- Chroma: An open-source embedding database designed for RAG applications
- Milvus: An open-source vector database for similarity search

When choosing a vector database, consider factors like scalability, query performance,
metadata filtering capabilities, and integration options.
"""

# Add metadata about the document
metadata = {
    "title": "Vector Databases Overview",
    "author": "AI Assistant",
    "category": "Technical",
    "tags": ["vector-db", "embeddings", "RAG"]
}

# Process the document
async def add_sample_document():
    result = await rag_agent.add_document(document_text, "sample-vector-db-doc", metadata)
    print(f"Added document with {len(result)} chunks")
    return result

# Run the async function
result = await add_sample_document()

## 2. Adding Documents from Files

The system can process various file types including PDF, Word, HTML, and text files:

In [ ]:
# Process a sample markdown file
async def add_sample_file():
    # Path to the sample file
    file_path = "../data/docs/rag_sample.md"
    
    # Add metadata
    metadata = {
        "category": "Educational",
        "tags": ["RAG", "tutorial", "AI"]
    }
    
    # Process the file
    result = await rag_agent.add_file(file_path, metadata)
    print(f"Processed file with {result['chunks_processed']} chunks")
    return result

# Run the async function
file_result = await add_sample_file()

## 3. Searching the Knowledge Base

You can search for documents semantically:

In [ ]:
# Search for information about vector databases
async def search_knowledge_base():
    query = "What are the best vector databases for RAG systems?"
    results = await rag_agent.search(query, k=3)
    
    print(f"Found {len(results)} relevant documents:\n")
    
    for i, result in enumerate(results):
        print(f"Result {i+1} (similarity: {result['similarity']:.2f})")
        print(f"Source: {result['metadata'].get('source', 'Unknown')}")
        
        # Print tags and category if available
        tags = result['metadata'].get('tags')
        if tags:
            print(f"Tags: {', '.join(tags)}")
            
        category = result['metadata'].get('category')
        if category:
            print(f"Category: {category}")
            
        # Print snippet of text
        text = result['text']
        print(f"\nText: {text[:200]}..." if len(text) > 200 else f"\nText: {text}")
        print("\n" + "-"*50 + "\n")
    
    return results

# Run the search
search_results = await search_knowledge_base()

## 4. Asking Questions with RAG

The RAG agent can answer questions based on the knowledge base:

In [ ]:
# Ask a question using RAG
async def ask_question():
    question = "Explain the components of a RAG architecture and why it's useful"
    
    # Get response from RAG agent
    response = await rag_agent.process({"query": question})
    
    print("Question:")
    print(question)
    print("\nAnswer:")
    print(response.text)
    
    return response

# Ask the question
rag_response = await ask_question()

## 5. Filtering Search Results

You can filter search results by metadata:

In [ ]:
# Search with metadata filters
async def filtered_search():
    query = "vector database options"
    
    # Filter by category
    filter_metadata = {
        "category": "Technical"
    }
    
    results = await rag_agent.search(
        query=query,
        k=5,
        filter_metadata=filter_metadata
    )
    
    print(f"Found {len(results)} matching Technical category results")
    
    for i, result in enumerate(results):
        print(f"\nResult {i+1} (similarity: {result['similarity']:.2f})")
        print(f"Category: {result['metadata'].get('category', 'None')}")
        print(f"Text snippet: {result['text'][:100]}...")
    
    return results

# Run filtered search
filtered_results = await filtered_search()

## 6. Saving the Vector Store

To persist the vector store for future use:

In [ ]:
# Save the vector store
async def save_vector_store():
    await enhanced_vector_store.save_index_async()
    print("Vector store saved successfully")

# Save the store
await save_vector_store()

## Conclusion

This demonstration shows how to use the RAG system for:
1. Adding documents from text and files
2. Searching the knowledge base semantically
3. Filtering results by metadata
4. Getting RAG-enhanced responses to questions

The system handles all the complexities of document processing, embedding generation, vector storage, and retrieval behind the scenes, providing a simple interface for building RAG-powered applications.